# IDEFICS: A Flamingo-based model, trained at scale for the community
# Finetuning Demo Notebook:


Credit: [Flamingo blog](https://www.deepmind.com/blog/tackling-multiple-tasks-with-a-single-visual-language-model)

This google colab notebook shows how to run predictions with the 4-bit quantized 🤗 [Idefics-9B model](https://huggingface.co/HuggingFaceM4/idefics-9b) and finetune it on a specific dataset.

[IDEFICS](https://huggingface.co/HuggingFaceM4/idefics-80b) is a multi-modal model based on the [Flamingo](https://arxiv.org/abs/2204.14198) architecture. It can take images and texts as input and return text outputs but it does not support image generation. \\
IDEFICS is built on top of two unimodal open-access pre-trained models to connect the two modalities. Newly initialized parameters in the form of Transformer blocks bridge the gap between the vision encoder and the language model. The model is trained on a mixture of image/text pairs and unstrucutred multimodal web documents. \\
The [finetuned versions](https://huggingface.co/HuggingFaceM4/idefics-80b-instruct) of IDEFICS behave like LLM chatbots while also understanding visual input. \\
You can play with the [demo here](https://huggingface.co/spaces/HuggingFaceM4/idefics_playground)

The code for this notebook was contributed to by *Léo Tronchon, Younes Belkada, and Stas Bekman*, the IDEFICS model has been contributed to by: *Lucile Saulnier, Léo Tronchon, Hugo Laurençon, Stas Bekman, Amanpreet Singh, Siddharth Karamcheti, and Victor Sanh*

# Install and import necessary libraries

In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

# Load quantized model
First get the quantized version of the model. This will allow us to use the 9B version of Idefics with a single 16GB gpu



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# checkpoint = "HuggingFaceM4/tiny-random-idefics"
checkpoint = "HuggingFaceM4/idefics-9b"

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=False)
# Simply take-off the quantization_config arg if you want to load the original model
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

If you print the model, you will see that all `nn.Linear` layers are in fact replaced by `bnb.nn.Linear4bit` layers.

In [ ]:
print(device)
print(model)

# Inference
Let's make a simple method to test the model's inference

In [ ]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)


Let's run prediction with the quantized model for the image below which pictures two kittens. \\
<img src="https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg" width="400"/>

In [ ]:
url = "https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg"
prompts = [
    # "Instruction: provide an answer to the question. Use the image to answer.\n",
    url,
    "Question: What's on the picture? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=5)


Now let's see how the model fares on pokemon knowledge before we try to finetune it further. \\
<img src="https://images.pokemontcg.io/pop6/2_hires.png" width="194"/>


In [ ]:
# check generation before finetuning

url = "https://images.pokemontcg.io/pop6/2_hires.png"
prompts = [
    url,
    "Question: What's on the picture? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=100)
# It looks like the model is already aware of pokemon - but it could be more specific, and less repetitive

# Finetuning dataset
Prepare the dataset that will be used for finetuning


In [ ]:
import requests
def is_image_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        content_type = response.headers.get('Content-Type')
        if content_type and 'image' in content_type.lower():
            return True
        else:
            return False
    except requests.RequestException as e:
        print(f"Error checking URL: {e}")
        return False

def convert_to_rgb(image):
    # `image.convert("RGB")` would only work for .jpg images, as it creates a wrong background
    # for transparent images. The call to `alpha_composite` handles this case
    if image.mode == "RGB":
        return image

    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite

def ds_transforms(example_batch):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std

    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std),
    ])

    prompts = []
    for i in range(len(example_batch['entity_name'])):
        # We split the captions to avoid having very long examples, which would require more GPU ram during training
        # caption = example_batch['caption'][i].split(".")[0]
        if is_image_url(example_batch['image_link'][i]):
          prompts.append(
              [
                  example_batch['image_link'][i],
                  # f"Question: What's the {example_batch['entity_name'][i]} of the . {caption}</s>",
                  f"Question: What's numerical {example_batch['entity_name'][i]} shown in the picture of the object, don't modify? Answer: {example_batch['entity_value'][i]}",
              ],
          )
        else:
          print("image doesn't exist here: ", example_batch['image_link'][i])

    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)

    inputs["labels"] = inputs["input_ids"]

    return inputs


# load and prepare dataset
# Load your CSV file into a DatasetDict
ds = load_dataset('csv', data_files={'train': '/content/train.csv'})
# ds = load_dataset("TheFusion21/PokemonCards")
ds = ds["train"].train_test_split(test_size=0.002)
train_ds = ds["train"]
eval_ds = ds["test"]
train_ds.set_transform(ds_transforms)
eval_ds.set_transform(ds_transforms)


In [ ]:
print(type(ds))

In [ ]:
train_ds

# LoRA
After specifying the low-rank adapters (LoRA) config, we load the PeftModel using the get_peft_model utility function

In [ ]:
checkpoint

In [ ]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

# Training
Finally, using the Hugging Face Trainer, we can finetune the model!

For the sake of the demo, we have set the max_steps at 40. That's about 0.05 epoch on this dataset, so feel free to tune further!

It has been reported that fine-tuning in mixed precision fp16 can lead to overflows. As such, we recommend training in mixed precision bf16 when possible.

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{model_name}-amazon-images",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=20,
    logging_steps=20,
    max_steps=40,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    report_to=None,
    optim="paged_adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
)

trainer.train()

In [ ]:
# Save the model weights
trainer.save_model(f"/content/{model_name}-amazon-images-ml")


## Inference from local model

In [ ]:
from transformers import AutoProcessor
checkpoint = "HuggingFaceM4/idefics-9b"
processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=False)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
from transformers import IdeficsForVisionText2Text
saved_model_path = f"/content/idefics-9b-amazon-images-ml"
# Load the processor (tokenizer, image processor, etc.)
# processor_ = AutoProcessor.from_pretrained(saved_model_path)

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

# Load the model
model_ = IdeficsForVisionText2Text.from_pretrained(saved_model_path, quantization_config=bnb_config).to("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model_.eval()

In [ ]:
ppt = [
    "https://m.media-amazon.com/images/I/51xvxFlRXoL.jpg",
    # f"Question: What's the {example_batch['entity_name'][i]} of the . {caption}</s>",
    "Question: What's numerical height shown in the picture of the objct, don't modify?",
]
check_inference(model, processor, ppt, max_new_tokens=100)

# Push your new model to the hub!


In [ ]:
!git config --global credential.helper store


In [ ]:
# Insert your "write" token. You should find it in the settings of your HF profile
!huggingface-cli login

## Upload Model to HuggingFace Hub
## Download and inference

In [ ]:
from huggingface_hub import HfApi, HfFolder

# Initialize the Hugging Face API
api = HfApi()
token = HfFolder.get_token()


# Upload the model files
api.upload_folder(
    folder_path="/content/idefics-9b-amazon-images-ml",  # Path to the folder containing model files
    repo_id="Ashu777/idefics-9b-amazon-image-captioning",
    repo_type="model",
    token=token
)


In [ ]:
!pip install bitsandbytes

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

In [ ]:
# checkpoint = "HuggingFaceM4/tiny-random-idefics"
checkpoint = "Ashu777/idefics-9b-amazon-image-captioning"

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

# processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=False)
# Simply take-off the quantization_config arg if you want to load the original model
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

In [ ]:
checkpoint = "HuggingFaceM4/idefics-9b"

processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=False)

In [ ]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

In [ ]:
ppt = [
    "https://m.media-amazon.com/images/I/71lxjbXw9bL.jpg",
    # f"Question: What's the {example_batch['entity_name'][i]} of the . {caption}</s>",
    "Question: What's weight",
]
check_inference(model, processor, ppt, max_new_tokens=100)

# inferencing at gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Gradio interface function
def run_inference(image_url, question):
    ppt = [image_url, f"Question: {question}"]
    return check_inference(model, processor, ppt, max_new_tokens=100)

# Launch the Gradio app
interface = gr.Interface(
    fn=run_inference,
    inputs=[gr.Textbox(label="Image URL"), gr.Textbox(label="Question")],
    outputs="text",
    title="Vision-Text Model Inference"
)

interface.launch()